In [ ]:
import numpy as np
import cv2 as cv
import os

### Calea folderelor de input si output

In [3]:
folder_test = "testare"     # folderul cu imaginile si fisierele de testare
folder_output = "output"    # folderul in care se vor scrie fisierele de output; trebuie sa fie deja creat
cale_poza_start = "auxiliare/01.jpg"    # calea catre prima poza auxiliara, fara piese pe tabla 

In [4]:
def extrage_careu(image):
    
    # Se aplica o masca HSV care elimina pixelii corespunzatori mesei
    frame = image.copy()
    frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    l = np.array([25, 0, 0])
    u = np.array([255, 255, 255])
    
    mask_table_hsv = cv.inRange(frame_hsv, l, u)        
    res = cv.bitwise_and(frame, frame, mask=mask_table_hsv)
    
    # Se aplica erosion pentru a elimina pixelii albi ramasi din masa
    res = cv.cvtColor(res, cv.COLOR_BGR2GRAY)
    kernel = np.ones((7, 7), np.uint8)
    res = cv.erode(res, kernel)
    
    # Se aplica un threshold, astfel incat aproape intreaga tabla devine alba, iar restul imaginii este neagra
    _, thresh = cv.threshold(res, 10, 255, cv.THRESH_BINARY)

    # Se determina colturile tablei utilizand codul din laborator
    contours, _ = cv.findContours(thresh,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    max_area = 0
   
    for i in range(len(contours)):
        if(len(contours[i]) >3):
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis = 1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left

    width = 1500
    height = 1500
    
    image_copy = image.copy()
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
    
    puzzle = np.array([top_left, top_right, bottom_right, bottom_left], dtype = "float32")
    destination_puzzle = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype = "float32")
    
    M = cv.getPerspectiveTransform(puzzle, destination_puzzle)
    result = cv.warpPerspective(image, M, (width, height))
    
    # Se extrage portiunea de joc la coordonate fixe din tabla mare
    img_noua = cv.resize(result[228:1273, 233:1267], (width, height))
    
    return img_noua

#### Functie pentru prelucrarea imaginilor

In [5]:
def prelucrare_det_pozitie(img, t=140):
    
    # Se aplica o masca HSV care inlatura majoritatea pixelilor colorati
    frame = img.copy()
    frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    l = np.array([75, 0, 0])
    u = np.array([120, 120, 255])
    mask_table_hsv = cv.inRange(frame_hsv, l, u)        
    res = cv.bitwise_and(frame, frame, mask=mask_table_hsv)
    
    # Se aplica un threshold astfel incat tabla este in mare masura neagra cu exceptia
    # pieselor de joc si a detaliilor deschise la culoare
    res = cv.cvtColor(res, cv.COLOR_BGR2GRAY)
    _, thresh = cv.threshold(res, t, 255, cv.THRESH_BINARY)
    
    return thresh

#### Prelucrarea imaginii de start pentru toate jocurile

In [6]:
img = cv.imread(cale_poza_start)
start = extrage_careu(img)
start = prelucrare_det_pozitie(start)

#### Functie pentru determinarea pozitiei in care s-a plasat piesa

In [7]:
# Functia primeste imaginea cu diferenta dintre doua imagini:
# cea pentru care se determina pozitia piesei adaugate si cea anterioara

# Se returneaza coordonatele in pixeli unde s-a pozitionat piesa
def gasire_coordonate(diff):
    inaltime, latime = 1500, 1500
    dim_chenar = 100

    max_diferenta1 = 0
    max_diferenta2 = 0
    coord_max1 = None
    coord_max2 = None
    
    for lin in range(0, inaltime - dim_chenar + 1, dim_chenar):
        for col in range(0, latime - dim_chenar + 1, dim_chenar):
            chenar_diferenta = diff[lin:lin+dim_chenar, col:col+dim_chenar]

            suma_diferente = np.sum(chenar_diferenta)

            if suma_diferente > max_diferenta1:
                max_diferenta2 = max_diferenta1
                coord_max2 = coord_max1

                max_diferenta1 = suma_diferente
                coord_max1 = (lin, col)

            elif suma_diferente > max_diferenta2:
                max_diferenta2 = suma_diferente
                coord_max2 = (lin, col)

    return coord_max1, coord_max2

#### Functie pentru convertirea coordonatelor in pixeli in pozitii ale jocului

In [8]:
def convert_to_excel_format(coord):
    litere_alfabet = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
    litera_coloana = litere_alfabet[coord[1]]
    excel_format = f"{coord[0]+1}{litera_coloana}"
    return excel_format

#### Setarea variabilelor pentru calcularea scorului

In [9]:
scoruri_tabla = np.array([
    [5, 0, 0, 4, 0, 0, 0, 3, 0, 0, 0, 4, 0, 0, 5],
    [0, 0, 3, 0, 0, 4, 0, 0, 0, 4, 0, 0, 3, 0, 0],
    [0, 3, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0],
    [4, 0, 0, 3, 0, 2, 0, 0, 0, 2, 0, 3, 0, 0, 4],
    [0, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 2, 0, 0],
    [0, 4, 0, 2, 0, 1, 0, 0, 0, 1, 0, 2, 0, 4, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 4, 0, 2, 0, 1, 0, 0, 0, 1, 0, 2, 0, 4, 0],
    [0, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 2, 0, 0],
    [4, 0, 0, 3, 0, 2, 0, 0, 0, 2, 0, 3, 0, 0, 4],
    [0, 3, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0],
    [0, 0, 3, 0, 0, 4, 0, 0, 0, 4, 0, 0, 3, 0, 0],
    [5, 0, 0, 4, 0, 0, 0, 3, 0, 0, 0, 4, 0, 0, 5]
])

In [10]:
traseu = [1, 2, 3, 4, 5, 6, 0, 2, 5, 3, 
          4, 6, 2, 2, 0, 3, 5, 4, 1, 6,
          2, 4, 5, 5, 0, 6, 3, 4, 2, 0,
          1, 5, 1, 3, 4, 4, 4, 5, 0, 6,
          3, 5, 4, 1, 3, 2, 0, 0, 1, 1,
          2, 3, 6, 3, 5, 4, 1, 0, 6, 6,
          5, 2, 1, 2, 5, 0, 3, 3, 5, 0,
          6, 1, 4, 0, 6, 3, 5, 1, 4, 1,
          4, 2, 6, 2, 3, 1, 6, 2, 0, 4,
          0, 1, 6, 4, 4, 1, 6, 6, 3, 0]

#### Citirea imaginilor din folderul de testare si prelucrarea lor

In [11]:
files = os.listdir(folder_test)
images = [[] for i in range(0,5)]

for file in files: 
    if file[-3:] == 'jpg':
        indice = int(file[0])-1
        img = cv.imread(os.path.join(folder_test, file))
        
        result = extrage_careu(img)
        result = prelucrare_det_pozitie(result)
        images[indice].append(result)

#### Sectiune de cod care determina piesele, pozitiile si scorul

In [12]:
for i in range(0, 5):
    
    # Citirea fisierului de mutari pentru fiecare joc
    cale_fisier_mutare = os.path.join(folder_test, f"{i+1}_mutari.txt")
    with open(cale_fisier_mutare, 'r') as file:
        mutari = [line.strip() for line in file.readlines()]
    
    scor = [0, 0] # scorul reprezinta si pozitia pe traseu

    for j in range(0, 20):
        
        # Se determina diferenta la nivel de pixel dintre imaginea actuala si cea anterioara
        if j == 0:
            diff = images[i][j] - start
        else:
            diff = images[i][j] - images[i][j-1]

        # Aceasta diferenta este folosita pentru a determina coordonatele piesei adaugate        
        coord1, coord2 = gasire_coordonate(diff)
        
        # Sortarea coordonatelor
        if (coord1[0] > coord2[0] or
            (coord1[0] == coord2[0] and coord1[1] > coord2[1])):
            aux = coord1
            coord1 = coord2
            coord2 = aux
            
        ### Determinarea numarului inscris pe piesele de domino    
        coordinates = (coord1, coord2)
        buline = []
        
        # Pentru fiecare jumatate din piesa se va determina numarul de buline de pe aceasta
        for poz in coordinates:
            lin1, col1 = poz[0], poz[1]             # coltul stanga sus
            lin2, col2 = poz[0]+100, poz[1]+100     # coltul dreapta jos
    
            # Pentru fiecare patrat se mai adauga 5 pixeli in toate directiile posibile
            # pentru cazurile in care piesele nu sunt asezate perfect
            lin1 = lin1 - 5 if lin1 != 0 else lin1
            col1 = col1 - 5 if col1 != 0 else col1
            
            lin2 = lin2 + 5 if lin2 != 1500 else lin2
            col2 = col2 + 5 if lin2 != 1500 else col2

            img = images[i][j][lin1:lin2, col1:col2].copy()
            poza = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
            
            # Prelucrari inainte de a aplica HoughCircles
            img = cv.medianBlur(img, ksize=5)
            img = cv.Canny(img, 50, 120)
            
            circles = cv.HoughCircles(
                img, cv.HOUGH_GRADIENT, 1.7, 7, 
                param1=75, param2=30, minRadius=8, maxRadius=14)
            
            if circles is None:
                nr_buline = 0
            else:
                _, nr_buline, _ = circles.shape
                
            # Daca din intamplare se detecteaza mai mult de 6 buline, atunci se limiteaza la 6
            nr_buline = 6 if nr_buline > 6 else nr_buline
    
            buline.append(nr_buline)
            
        
        ### Mentinerea scorului
    
        # Coordonatele folosite pentru tabla de joc
        coord1 = tuple(coord // 100 for coord in coord1)
        coord2 = tuple(coord // 100 for coord in coord2)
        
        # Coordonatele scrise in fisier
        excel_format_max1 = convert_to_excel_format(coord1)
        excel_format_max2 = convert_to_excel_format(coord2)
        
        jucator_curent = int(mutari[j][-1]) - 1
        jucator_opus = 1 - jucator_curent
    
        # Se adauga scorul mutarii
        scor_mutare = scoruri_tabla[coord1[0]][coord1[1]] + scoruri_tabla[coord2[0]][coord2[1]]
        
        # Se verifica daca e piesa dubla, caz in care scorul se dubleaza
        if buline[0] == buline[1]:
            scor_mutare *= 2
        
        # Se adauga bonusurile pentru ambii jucatori, daca e cazul
        if scor[jucator_curent] > 0 and traseu[scor[jucator_curent] - 1] in buline:
            scor_mutare += 3
        
        if scor[jucator_opus] > 0 and traseu[scor[jucator_opus] - 1] in buline:
            scor[jucator_opus] += 3
         
        scor[jucator_curent] += scor_mutare
        
        # Se scrie rezultatul in fisierul de output
        cale_fisier_output = os.path.join(folder_output, f"{i+1}_{str(j+1).zfill(2)}.txt")
        
        with open(cale_fisier_output, "w") as file:
            file.write(f"{excel_format_max1} {buline[0]}\n"
                       f"{excel_format_max2} {buline[1]}\n"
                       f"{scor_mutare}")